In [ ]:
# CSCI 332 - Advanced Algorithms and Data Structures
# Professor: Jordan Malof
# Date: 11/04/2023
# Author: Bogdan Bošković
# Description: This program implements Kruskal's algorithm for finding
# the minimum spanning tree of a weighted graph, utilizing a union-find
# data structure to keep track of the connected components of the graph.
# An implementation of merge sort is also required.
import numpy as np

class kruskalClass():
  def __init__(self):
    pass

  def findMinimumSpanningTree(self, A):
    # A is an NxN numpy array representing the adjacency matrix of a graph
    # 0's represent no edge, positive numbers represent the weight of an edge
    n = A.shape[0]  # Number of nodes
    T = np.zeros((n, n))
    # create a list of edges (weight, u, v) from the adjacency matrix
    edges = []
    for i in range(n):
      for j in range(i + 1, n):  # consider each edge only once
        if A[i][j] != 0:  # if there is an edge, add it to the list
          edges.append((A[i][j], i, j))

    # we don't even need to use the old indices, really? not cool, man.
    edges, indices = self.mergesort(edges) # sort the edges by weight

    uf = self.makeUnionFind(n)  # create a union-find data structure
    for weight, u, v in edges:
      # if no cycle, add edge to T and union the sets that u and v belong to
      if self.find(uf, u) != self.find(uf, v):
        T[u][v] = weight # not T[u][v] = T[v][u] = weight, because upper triangle thing
        uf = self.union(uf, u, v)

    return T  # NxN numpy array representing the minimum spanning tree of A (upper right triangular matrix).

  def find(self, u, v):
    # u is a union-find data structure
    # v is a numerical index for a graph node
    # set s to the label of the set to which v belongs
    if u[v][0] != v:  
      # recursively find the label of the set to which the parent of v belongs
      u[v][0] = self.find(u, u[v][0])

    return u[v][0] # label of the set to which v belongs

  # here, N is the number of nodes in a graph
  def makeUnionFind(self, N):
    u = {}
    # in u, each node has a 2-d numpy array associated with it. The first entry is a pointer.
    # if the pointer and the key are the same, then the name of the set to which the node belongs is itself.
    # The second entry in each numpy array is the count of the number of pointers that point to the given node
    for i in range(N):
      u[i] = np.array([i, 1])

    return u # 1xN python dict, wiith keys as numerical labels for the nodes, and values numpy arrays with 0th entry being a pointer to a node in the same dict
  
  def union(self, u, s1, s2):
    # union the sets that s1 and s2 belong to and return the updated union-find structure
    # find the roots of the trees that s1 and s2 belong to
    root1 = self.find(u, s1)
    root2 = self.find(u, s2)
    if root1 != root2:
        # if the trees are different sizes, make the smaller one a subtree of the larger one
        if u[root1][1] < u[root2][1]:
            # set pointer of root1 to root2
            u[root1][0] = root2
            # increment the count of the number of pointers that point to root2
            u[root2][1] += u[root1][1]
        elif u[root1][1] > u[root2][1]:
            # set pointer of root2 to root1
            u[root2][0] = root1
            # increment the count of the number of pointers that point to root1
            u[root1][1] += u[root2][1]
        else:
            # if both trees are the same size, make one a subtree of the other
            u[root2][0] = root1
            # increment the count of the number of pointers that point to root1
            u[root1][1] += u[root2][1]

    return u

  # merge sort implementation
  def mergesort(self, a):
    # Pair each element with its original index, for later
    a = [(a[i], i) for i in range(len(a))]
    # inner function to split the input array recursively into tiny arrays
    def divide(a):
      if len(a) <= 1:
        return a
      midpoint = len(a) // 2
      left = divide(a[:midpoint])
      right = divide(a[midpoint:])
      # call the merge inner function below to merge the tiny arrays
      return merge(left, right)
    # inner function for sorting and merging the tiny arrays
    def merge(left, right):
      # init counters and results list
      result = []
      i = j = 0
      # while there are elements in both arrays, compare the values
      while i < len(left) and j < len(right):
        # if the left i VALUE is smaller than right j VALUE, add it to results
        if left[i][0] < right[j][0]:
          result.append(left[i])
          # increment the left comparison counter
          i += 1
        else:
          # if it's larger, then add the right j value to results
          result.append(right[j])
          # increment the right comparison counter
          j += 1
      # if there are any elements left in the left and right arrays, add them to results    
      result.extend(left[i:])
      result.extend(right[j:])
      return result
    # Perform the merge sort
    sorted_pairs = divide(a)
    # separate the sorted pairs into two arrays, one for the values and one for the indices
    b = [pair[0] for pair in sorted_pairs]
    inds = [pair[1] for pair in sorted_pairs]

    return [b, inds]

In [47]:
####################################################################################################

krusky = kruskalClass()

n = 5
u = krusky.makeUnionFind(n)
print(f"{u}")
s1 = krusky.find(u, 2)
print(f"{s1}")
s2 = krusky.find(u, 4)
print(f"{s2}")

u1 = krusky.union(u, krusky.find(u, 0), krusky.find(u, 1))
print(f"{krusky.find(u1, 0)}")
print(f"{u1}")

u2 = krusky.union(u, krusky.find(u1, 0), krusky.find(u1, 2))
print(f"{u2}")

# Create a test matrix
A = np.array([[0, 8, 0, 3],
 [0, 0, 2, 5],
 [0, 0, 0, 6],
 [0, 0, 0, 0]])
#Use code to generate a MST
T = krusky.findMinimumSpanningTree(A)

print(f"{T}")
print(f"{type(T)}\n\n\n\n\n")


A = np.array([[0, 10, 20, 50, 100, 32, 29, 15, 60, 70, 80, 90, 990],
              [0,  0,  1,  0,   0,  0, 17,  1,  1,  1, 16,  0,   0],
              [0,  0,  0,  0,   3,  0,  0,  1,  8,  0,  4,  0,   0],
              [0,  0,  0,  0,   0,  1,  5,  0,  0,  1,  0,  2,   0],
              [0,  0,  0,  0,   0, 21,  0,  0,  0,  0,  9,  0,   0],
              [0,  0,  0,  0,   0,  0,  0,  0,  0, 12,  0,  1,   0],
              [0,  0,  0,  0,   0,  0,  0,  0,  11, 0,  0,  0,   0],
              [0,  0,  0,  0,   0,  0,  0,  0,   0, 0,  1,  31,  0],
              [0,  0,  0,  0,   0,  0,  0,  0,   0, 0,  9,  1,   0],
              [0,  0,  0,  0,   0,  0,  0,  0,   0, 0,  0,  1,   1],
              [0,  0,  0,  0,   0,  0,  0,  0,   0, 0,  0,  6,   1],
              [0,  0,  0,  0,   0,  0,  0,  0,   0, 0,  0,  0,   14],
              [0,  0,  0,  0,   0,  0,  0,  0,   0, 0,  0,  0,   0]])
T = krusky.findMinimumSpanningTree(A)

#Print the MST
print(T)
print(T.sum())

{0: array([0, 1]), 1: array([1, 1]), 2: array([2, 1]), 3: array([3, 1]), 4: array([4, 1])}
2
4
0
{0: array([0, 2]), 1: array([0, 1]), 2: array([2, 1]), 3: array([3, 1]), 4: array([4, 1])}
{0: array([0, 3]), 1: array([0, 1]), 2: array([0, 1]), 3: array([3, 1]), 4: array([4, 1])}
[[0. 0. 0. 3.]
 [0. 0. 2. 5.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<class 'numpy.ndarray'>





[[ 0. 10.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  5.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 